In [1]:
from nustar_pysolar import planning, io
import astropy.units as u
import warnings
warnings.filterwarnings('ignore')

The 'recommended_units' attribute is deprecated since 3.0 and may be removed
in a future version. Its main use, of representing angles in degrees in frames,
is now done automatically in frames. Further overrides are discouraged but can
be done using a frame's ``frame_specific_representation_info``.
 [astropy.coordinates.representation]


# Download the list of occultation periods from the MOC at Berkeley.

## Note that the occultation periods typically only are stored at Berkeley for the *future* and not for the past. So this is only really useful for observation planning.

In [2]:
fname = io.download_occultation_times(outdir='../data/')
print(fname)

../data/NUSTAR.2018_268.SHADOW_ANALYSIS.txt


# Download the NuSTAR TLE archive.

This contains every two-line element (TLE) that we've received for the whole mission. We'll expand on how to use this later.

The `times`, `line1`, and `line2` elements are now the TLE elements for each epoch.

In [3]:
tlefile = io.download_tle(outdir='../data')
print(tlefile)
times, line1, line2 = io.read_tle_file(tlefile)

../data/NuSTAR.tle


# Here is where we define the observing window that we want to use.

Note that tstart and tend must be in the future otherwise you won't find any occultation times and sunlight_periods will return an error.

In [4]:
tstart = '2018-09-27T12:00:00'
tend = '2018-09-29T12:10:00'
orbits = planning.sunlight_periods(fname, tstart, tend)

# We want to know how to orient NuSTAR for the Sun.

We can more or less pick any angle that we want. But this angle has to be specified a little in advance so that the NuSTAR SOC can plan the "slew in" maneuvers. Below puts DET0 in the top left corner (north-east with respect to RA/Dec coordinates).

### This is what you tell the SOC you want the "Sky PA angle" to be.

In [ ]:
pa = planning.get_nustar_roll(tstart, 0)
print("NuSTAR Roll angle for Det0 in NE quadrant: {}".format(pa))

# Set up the offset you want to use here:

The first element is the direction +WEST of the center of the Sun, the second is the offset +NORTH of the center of the Sun.

If you want multiple pointing locations you can either specify an array of offsets or do this "by hand" below.

In [5]:
offset = [0., 0.]*u.arcsec

# Loop over each orbit and correct the pointing for the same heliocentric pointing position.

Note that you may want to update the pointing for solar rotation. That's up to the user to decide and is not done here.

In [6]:
for ind, orbit in enumerate(orbits):
    midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
    sky_pos = planning.get_skyfield_position(midTime, offset, parallax_correction=True)
    print("Orbit: {}".format(ind))
    print("Orbit start: {} Orbit end: {}".format(orbit[0].isoformat(), orbit[1].isoformat()))
    print('Aim time: {} RA (deg): {} Dec (deg): {}'.format(midTime.isoformat(), sky_pos[0], sky_pos[1]))
    print("")

[#################################] 100% deltat.data
[#################################] 100% Leap_Second.dat


Orbit: 0
Orbit start: 2018-09-27T11:47:30 Orbit end: 2018-09-27T12:48:50
Aim time: 2018-09-27T12:18:10 RA (deg): 183.75361356568973 deg Dec (deg): -1.625719371459118 deg

Orbit: 1
Orbit start: 2018-09-27T13:24:10 Orbit end: 2018-09-27T14:25:30
Aim time: 2018-09-27T13:54:50 RA (deg): 183.8140598312915 deg Dec (deg): -1.651848616119325 deg

Orbit: 2
Orbit start: 2018-09-27T15:00:50 Orbit end: 2018-09-27T16:02:10
Aim time: 2018-09-27T15:31:30 RA (deg): 183.87451015897653 deg Dec (deg): -1.6779770391065716 deg

Orbit: 3
Orbit start: 2018-09-27T16:37:30 Orbit end: 2018-09-27T17:38:50
Aim time: 2018-09-27T17:08:10 RA (deg): 183.93496457952233 deg Dec (deg): -1.7041046132672741 deg

Orbit: 4
Orbit start: 2018-09-27T18:14:10 Orbit end: 2018-09-27T19:15:20
Aim time: 2018-09-27T18:44:45 RA (deg): 183.9953853799726 deg Dec (deg): -1.730207465641824 deg

Orbit: 5
Orbit start: 2018-09-27T19:50:50 Orbit end: 2018-09-27T20:52:00
Aim time: 2018-09-27T20:21:25 RA (deg): 184.05584807362263 deg Dec (deg)

# This is where you actually make the Mosaic

In [9]:
# Just use the first orbit...or choose one. This may download a ton of deltat.preds, which is a known 
# bug to be fixed.

orbit = orbits[20]
planning.make_mosaic(orbit, write_output=True, make_regions=True)

('Step PA', <Quantity 115.78189166 deg>)
Orbit start: 2018-09-28T20:00:40 Orbit end: 2018-09-28T21:02:00
('Dwell per position:', datetime.timedelta(0, 147, 200000))

NuSTAR Roll Angle to get DET0 in top right 25.78 deg

